In [17]:
import pandas as pd
import numpy as np
import re


In [2]:
df = pd.read_csv(r'../../../data/data.csv')

In [15]:
df

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
0,NaN,322111,3F - UNITED FEDERATION OF DANISH WORKERS,@3f.dk,NaN,NaN
1,NaN,411480,3F - UNITED FEDERATION OF WORKERS IN DENMARK,@3f.dk,NaN,NaN
2,NaN,493788,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,NaN,NaN
3,NaN,511603,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,NaN,NaN
4,NaN,71936,AGORIA - BELGIAN FEDERATION FOR THE TECHNOLOGY...,NaN,AGORIA,NaN
...,...,...,...,...,...,...
674,NaN,480371,YRITTÄJÄT/FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,NaN,NaN
675,NaN,291768,ZAW (GERMAN ADVERTISING FEDERATION),@zaw.de,ZAW,NaN
676,NaN,526538,ZDH- CONFEDERATION OF GERMAN SKILLED CRAFTS AN...,@zdh.de,NaN,NaN
677,NaN,496681,ZDH GERMAN CONFEDERATION OF SKILLED CRAFTS AND...,@zdh.de,NaN,NaN


In [19]:
delimiters = " ", "/", "-"
regex_pattern = '|'.join(map(re.escape, delimiters))
regex_pattern

'\\ |/|\\-'

In [29]:
df['Acronym'] = df.Account.apply(lambda x: re.split(regex_pattern, x)[0])

In [33]:
df

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
0,NaN,322111,3F - UNITED FEDERATION OF DANISH WORKERS,@3f.dk,3F,NaN
1,NaN,411480,3F - UNITED FEDERATION OF WORKERS IN DENMARK,@3f.dk,3F,NaN
2,NaN,493788,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,A.I.S.E.,NaN
3,NaN,511603,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,A.I.S.E.,NaN
4,NaN,71936,AGORIA - BELGIAN FEDERATION FOR THE TECHNOLOGY...,NaN,AGORIA,NaN
...,...,...,...,...,...,...
674,NaN,480371,YRITTÄJÄT/FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,YRITTÄJÄT,NaN
675,NaN,291768,ZAW (GERMAN ADVERTISING FEDERATION),@zaw.de,ZAW,NaN
676,NaN,526538,ZDH- CONFEDERATION OF GERMAN SKILLED CRAFTS AN...,@zdh.de,ZDH,NaN
677,NaN,496681,ZDH GERMAN CONFEDERATION OF SKILLED CRAFTS AND...,@zdh.de,ZDH,NaN


In [160]:
acc_list = df.groupby(by='Acronym')['Account'].agg(lambda x: x.value_counts(ascending=False).index[0] 
                                                    if x.value_counts(ascending=False)[0] > 1 
                                                    else x.value_counts(ascending=False).index[:])

In [135]:
df.dtypes

MasterID                    float64
AccountID                     int64
Account                      object
Email domain                 object
Acronym                      object
[Bonus] Confidence score    float64
dtype: object

In [193]:
for acc in acc_list:
    if type(acc) is str:
        acronym = df.loc[(df.Account == acc)].Acronym.values[0]
        id = df.loc[(df.Account == acc)].AccountID.values[0]
        df.loc[(df.Acronym == acronym), 'MasterID'] = id
    else:
        acronym = df.loc[(df.Account == acc[0])].Acronym.values[0]
        id = df.loc[(df.Acronym == acronym)].AccountID.sort_values(ascending=True).reset_index(drop=True)[0]
        df.loc[(df.Acronym == acronym), 'MasterID'] = id

this is a list
3F - UNITED FEDERATION OF DANISH WORKERS
3F
this is a list
A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETERGENTS AND HOUSEHOLD MAINTENANCE PRODUCTS
A.I.S.E.
this is a list
AGRICULTURAL FEDERATION OF THE AZORES
AGRICULTURAL
this is a list
AMERICAN FEDERATION OF TEACHERS
AMERICAN
this is a list
BANKING & PAYMENTS FEDERATION IRELAND - BRUSSELS
BANKING
this is a list
BDI - FEDERATION OF GERMAN INDUSTRIES
BDI
this is a list
BELGIAN CONFEDERATION OF THE DAIRY INDUSTRY
BELGIAN
this is a list
BRITISH COATINGS FEDERATION
BRITISH
this is a list
CANADIAN FEDERATION OF INDEPENDENT BUSINESS
CANADIAN
this is a list
CEOE (SPANISH CONFEDERATION OF EMPLOYERS)
CEOE
this is a list
CHEMICAL INDUSTRY FEDERATION OF FINLAND
CHEMICAL
this is a list
CIP - CONFEDERATION OF PORTUGUESE BUSINESS
CIP
this is a list
CONCORDIA CONFEDERATION
CONCORDIA
this is a list
CONFÉDÉRATION DES PETITES ET MOYENNES ENTREPRISES
CONFÉDÉRATION
this is a list
DANISH CONFEDERATION OF TRADE UNIONS
DANISH
this is a list
DF

In [195]:
df.MasterID = df.MasterID.astype(np.int64)

In [198]:
df

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
0,322111,322111,3F - UNITED FEDERATION OF DANISH WORKERS,@3f.dk,3F,NaN
1,322111,411480,3F - UNITED FEDERATION OF WORKERS IN DENMARK,@3f.dk,3F,NaN
2,493788,493788,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,A.I.S.E.,NaN
3,493788,511603,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,A.I.S.E.,NaN
4,71936,71936,AGORIA - BELGIAN FEDERATION FOR THE TECHNOLOGY...,NaN,AGORIA,NaN
...,...,...,...,...,...,...
674,480371,480371,YRITTÄJÄT/FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,YRITTÄJÄT,NaN
675,291768,291768,ZAW (GERMAN ADVERTISING FEDERATION),@zaw.de,ZAW,NaN
676,496681,526538,ZDH- CONFEDERATION OF GERMAN SKILLED CRAFTS AN...,@zdh.de,ZDH,NaN
677,496681,496681,ZDH GERMAN CONFEDERATION OF SKILLED CRAFTS AND...,@zdh.de,ZDH,NaN
